In [1]:
from hpmetad import HPMetaD
import hoomd
import gsd.hoomd
import numpy as np
from hoomd import hpmc
import math
import itertools
import time
import coxeter

In [2]:
# Define shape
alpha_a = 0.1
alpha_c = 0.3

a = alpha_a * (3 - 1) + 1
c = alpha_c * (3 - 1) + 1
target_pf = 0.6

name = f'shape_a_{int(alpha_a * 100)}_c_{int(alpha_c * 100)}'
shape = coxeter.families.Family323Plus.get_shape(a=a, c=c)
shape.volume = 1
shape.center = [0, 0, 0]

verts = shape.vertices.tolist()

In [3]:
# Simulation parameters
run_params = dict(N_random = 1e3, N_compress = 1e3, N_tune = 2e3, N_eq = 1e6)
fname = f'{name}_pf_{target_pf}'
comm = hoomd.communicator.Communicator()
cpu = hoomd.device.CPU(communicator=comm, notice_level=2)

In [4]:
# Create an initial configuration at a low density
N_particles = 50
K = math.ceil(N_particles**(1 / 3))
spacing = 2.5 # > particle size
L = K * spacing
x = np.linspace(-L / 2,
                L / 2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))
position = position[0 : N_particles]
orientation = [[1, 0, 0, 0]] * N_particles

snapshot = gsd.hoomd.Frame()
snapshot.particles.N = N_particles
snapshot.particles.position = position
snapshot.particles.orientation = orientation
snapshot.particles.types = [name]
snapshot.configuration.box = [L, L, L, 0, 0, 0]
with gsd.hoomd.open('./data/init_config.gsd', mode='w') as f:
    f.append(snapshot)

In [5]:
# Randomize the system and compress to target density while tuning the particles' move sizes
# Save the trajectory and output
seed = np.random.randint(np.random.randint(10000))
sim = hoomd.Simulation(device=cpu, seed=seed)
sim.create_state_from_gsd(filename='./data/init_config.gsd')

mc = hpmc.integrate.ConvexPolyhedron(default_d=0.2, default_a=0.2)
mc.shape[name] = dict(vertices=verts)

sim.operations.integrator = mc
mc_tuner = hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                           target=0.2,
                                           max_translation_move=1,
                                           max_rotation_move=2,
                                           trigger=hoomd.trigger.And([
        hoomd.trigger.Periodic(10),
        hoomd.trigger.Before(int(run_params['N_random'] + run_params['N_compress'] + run_params['N_tune'])),
        hoomd.trigger.After(int(run_params['N_random'])),
]))

sim.operations.tuners.append(mc_tuner)

target_box = hoomd.variant.box.InverseVolumeRamp(initial_box=sim.state.box,
                                                 final_volume=sim.state.N_particles * shape.volume / target_pf,
                                                 t_start=int(run_params['N_random']), 
                                                 t_ramp=int(run_params['N_compress']))
qc = hpmc.update.QuickCompress(10, target_box)
sim.operations.updaters.append(qc)

dump_period = 1e2
log_period = 5e2

table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'seed', 'walltime'])
p_types = sim.state.particle_types

def add_quantities_to_logger(custom_logger):
    custom_logger[('overlaps', )] = (lambda: mc.overlaps, 'scalar')
    custom_logger[('box_volume', )] = (lambda: sim.state.box.volume, 'scalar')

    # Log MC params
    for p_type in p_types:
        custom_logger[(f'mc_a_{p_type}',)] = (lambda: mc.a[p_type], 'scalar')
        custom_logger[(f'mc_d_{p_type}',)] = (lambda: mc.d[p_type], 'scalar')

    custom_logger[('translate_move_acceptance_ratio', )] = (
        lambda: mc.translate_moves[0] / sum(mc.translate_moves), 'scalar')
    custom_logger[('rotate_move_acceptance_ratio', )] = (
        lambda: mc.rotate_moves[0] / sum(mc.rotate_moves), 'scalar')

add_quantities_to_logger(table_logger)
output_file = open(f'./data/output_{fname}.txt', mode='w', newline='\n')
table = hoomd.write.Table(output=output_file,
                          trigger=hoomd.trigger.Periodic(period=int(log_period)),
                          logger=table_logger)
sim.operations.writers.append(table)

logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

traj_writer = hoomd.write.GSD(filename=f'./data/init_nvt_{fname}.gsd',
                            trigger=hoomd.trigger.Periodic(int(dump_period)),
                            mode='wb',
                            dynamic=['property'],
                            logger=logger)
sim.operations.writers.append(traj_writer)

while (
    sim.timestep < target_box.t_ramp + target_box.t_start + run_params['N_tune'] or
    not qc.complete):
    sim.run(dump_period)

sim.operations.writers.remove(traj_writer)
sim.operations.tuners.remove(mc_tuner)
sim.operations.updaters.remove(qc)

In [6]:
# Run the HPMetaD simulation
# Save Metadynamics data to an hdf5 file and a txt file
# Save trajectory data to a gsd file

metad_stride = 10

sigma = 0.002
gamma = 10
init_height = 0.3

cv_min = 0.0
cv_max = 0.5

bins = 30

colvar_mode = 'steinhardt'
colvar_params = dict(neighbor={'r_max': 2.0}, l=6, average=False, wl=False, weighted=False, wl_normalize=False)


hpmetad_action = HPMetaD(
    sim=sim,
    colvar_mode=colvar_mode,
    colvar_params = colvar_params,
    kT=1,
    init_height=init_height,
    metad_stride=metad_stride,
    sigma=sigma,
    gamma=gamma,
    cv_min=cv_min,
    cv_max=cv_max,
    bins=bins,
    seed=seed,
)

op = hoomd.update.CustomUpdater(action=hpmetad_action, trigger=1)
sim.operations += op
sim.operations += hpmetad_action.get_writer('./data/WTMetaD_log.hdf5')

traj_writer = hoomd.write.GSD(filename=f'./data/traj_{fname}.gsd',
                              trigger=hoomd.trigger.Periodic(int(dump_period), phase=1),
                              mode='wb',
                              dynamic=['property'],
                              logger=logger)
sim.operations.writers.append(traj_writer)

total_timestep = run_params['N_eq'] + run_params['N_random'] + run_params['N_compress'] + run_params['N_tune']
while sim.timestep < total_timestep:
    step_size = int(total_timestep - sim.timestep)
    sim.run(min(1e2, step_size))
    hpmetad_action.save_history('./data/WTMetaD_history.txt')
    for writer in sim.operations.writers:
        if hasattr(writer, "flush"):
            writer.flush()

hpmetad_action.save_configuration(config_fn=f'./data/restart_{fname}.gsd', logger=logger)